In [2]:
import os
import cv2  

def extract_frames(video_path, output_folder):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    vidcap = cv2.VideoCapture(video_path)
    success, image = vidcap.read()
    count = 0

    while success:
        frame_path = os.path.join(output_folder, f"frame{count:04d}.jpg")
        cv2.imwrite(frame_path, image)
        success, image = vidcap.read()
        count += 1

    vidcap.release()

def process_videos_in_folder(input_folder, output_base_folder):
    for root, dirs, files in os.walk(input_folder):
        for file in files:
            if file.endswith('.webm'):
                video_path = os.path.join(root, file)
                video_name = os.path.splitext(file)[0]  # Mendapatkan nama video tanpa ekstensi
                output_folder = os.path.join(output_base_folder, video_name)
                extract_frames(video_path, output_folder)

# Penggunaan
input_folder = "C:/Users/askas/Downloads/Data Mentah/P"
output_base_folder = "C:/Users/askas/Downloads/Dataset/P"
process_videos_in_folder(input_folder, output_base_folder)


KeyboardInterrupt: 

In [31]:
import os
import cv2

def extract_frames(video_path, output_folder, target_size=(640, 480), max_file_size_kb=100):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    vidcap = cv2.VideoCapture(video_path)
    success, image = vidcap.read()
    count = 0

    while success:
        # Resize the image to target size
        resized_image = cv2.resize(image, target_size)

        # Initialize compression quality
        compression_quality = 95  # Start with high quality
        encode_param = [int(cv2.IMWRITE_JPEG_QUALITY), compression_quality]

        # Compress image and check size
        while True:
            is_success, encoded_image = cv2.imencode('.jpg', resized_image, encode_param)
            if is_success:
                if len(encoded_image) / 1024 <= max_file_size_kb or compression_quality <= 10:
                    break
                compression_quality -= 5  # Reduce quality to reduce size
                encode_param = [int(cv2.IMWRITE_JPEG_QUALITY), compression_quality]

        # Save the compressed image
        frame_path = os.path.join(output_folder, f"frame{count:04d}.jpg")
        with open(frame_path, 'wb') as f:
            f.write(encoded_image)

        success, image = vidcap.read()
        count += 1

    vidcap.release()

def process_videos_in_folder(input_folder, output_base_folder, target_size=(640, 480), max_file_size_kb=100):
    for root, dirs, files in os.walk(input_folder):
        for file in files:
            if file.endswith('.webm'):
                video_path = os.path.join(root, file)
                video_name = os.path.splitext(file)[0]
                output_folder = os.path.join(output_base_folder, video_name)
                extract_frames(video_path, output_folder, target_size, max_file_size_kb)

# Contoh penggunaan
input_folder = "C:/Users/askas/Downloads/Data Mentah/z"
output_base_folder = "C:/Users/askas/Downloads/Dataset/z"
process_videos_in_folder(input_folder, output_base_folder)

In [12]:
### Langkah 1: Mengimpor Library yang Dibutuhkan
import os
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, LSTM, TimeDistributed
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [16]:
def load_dataset(data_path):
    data = []
    labels = []
    label_map = {chr(i): idx for idx, i in enumerate(range(ord('A'), ord('Z')+1))}
    
    print(f"Label map: {label_map}")
    
    for root, _, files in os.walk(data_path):
        # Ganti backslash menjadi forward slash sebelum membaca file
        root = root.replace('\\', '/')
        print(f"Reading folder: {root}")
        folder_name = os.path.basename(root)
        if folder_name in label_map:
            label = label_map[folder_name]
            for file_name in files:
                file_path = os.path.join(root, file_name)
                if file_name.lower().endswith(('.png', '.jpg', '.jpeg')):
                    image = cv2.imread(file_path)
                    if image is not None:
                        image = cv2.resize(image, (64, 64))  # Ubah ukuran gambar jika diperlukan
                        data.append(image)
                        labels.append(label)
                    else:
                        print(f"Warning: Gagal membaca gambar {file_path}")
                else:
                    print(f"Warning: {file_path} bukan file gambar yang didukung")

    return np.array(data), np.array(labels)

data_path = 'C:/Users/askas/Downloads/Dataset'
X, y = load_dataset(data_path)

# Debugging untuk memastikan data telah dimuat dengan benar
print(f"Shape of X: {X.shape}")
print(f"Shape of y: {y.shape}")

# Normalisasi gambar
X = X / 255.0

# Ubah label menjadi one-hot encoding
y = to_categorical(y, num_classes=26)

# Debugging untuk memastikan normalisasi dan one-hot encoding berhasil
print(f"First 5 labels (one-hot encoded):\n {y[:5]}")


Label map: {'A': 0, 'B': 1, 'C': 2, 'D': 3, 'E': 4, 'F': 5, 'G': 6, 'H': 7, 'I': 8, 'J': 9, 'K': 10, 'L': 11, 'M': 12, 'N': 13, 'O': 14, 'P': 15, 'Q': 16, 'R': 17, 'S': 18, 'T': 19, 'U': 20, 'V': 21, 'W': 22, 'X': 23, 'Y': 24, 'Z': 25}
Reading folder: C:/Users/askas/Downloads/Dataset
Reading folder: C:/Users/askas/Downloads/Dataset/A
Reading folder: C:/Users/askas/Downloads/Dataset/A/A
Reading folder: C:/Users/askas/Downloads/Dataset/A/A I D S
Reading folder: C:/Users/askas/Downloads/Dataset/A/Aba
Reading folder: C:/Users/askas/Downloads/Dataset/A/Aba-aba
Reading folder: C:/Users/askas/Downloads/Dataset/A/Abad
Reading folder: C:/Users/askas/Downloads/Dataset/A/Abadi
Reading folder: C:/Users/askas/Downloads/Dataset/A/Abai
Reading folder: C:/Users/askas/Downloads/Dataset/A/Abang
Reading folder: C:/Users/askas/Downloads/Dataset/A/Abdi
Reading folder: C:/Users/askas/Downloads/Dataset/A/Abjad
Reading folder: C:/Users/askas/Downloads/Dataset/A/Absen
Reading folder: C:/Users/askas/Downloads/D

In [17]:
### Langkah 3: Membagi Dataset menjadi Train dan Test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [18]:
### Langkah 4: Membangun Model
def create_model(input_shape, num_classes):
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))
    return model

input_shape = (64, 64, 3)
num_classes = 26
model = create_model(input_shape, num_classes)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

c:\Users\askas\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [19]:
### Langkah 5: Melatih Mode
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)

Epoch 1/10
84/84 ━━━━━━━━━━━━━━━━━━━━ 7s 48ms/step - accuracy: 0.1138 - loss: 3.1537 - val_accuracy: 0.4158 - val_loss: 1.9812
Epoch 2/10
84/84 ━━━━━━━━━━━━━━━━━━━━ 4s 42ms/step - accuracy: 0.5173 - loss: 1.6468 - val_accuracy: 0.6632 - val_loss: 1.0433
Epoch 3/10
84/84 ━━━━━━━━━━━━━━━━━━━━ 3s 39ms/step - accuracy: 0.7611 - loss: 0.8011 - val_accuracy: 0.8405 - val_loss: 0.5589
Epoch 4/10
84/84 ━━━━━━━━━━━━━━━━━━━━ 3s 40ms/step - accuracy: 0.8550 - loss: 0.4624 - val_accuracy: 0.9091 - val_loss: 0.3181
Epoch 5/10
84/84 ━━━━━━━━━━━━━━━━━━━━ 3s 40ms/step - accuracy: 0.9071 - loss: 0.3070 - val_accuracy: 0.8808 - val_loss: 0.3356
Epoch 6/10
84/84 ━━━━━━━━━━━━━━━━━━━━ 3s 40ms/step - accuracy: 0.9185 - loss: 0.2537 - val_accuracy: 0.9538 - val_loss: 0.1705
Epoch 7/10
84/84 ━━━━━━━━━━━━━━━━━━━━ 3s 40ms/step - accuracy: 0.9458 - loss: 0.1802 - val_accuracy: 0.9285 - val_loss: 0.2239
Epoch 8/10
84/84 ━━━━━━━━━━━━━━━━━━━━ 3s 40ms/step - accuracy: 0.9469 - loss: 0.1584 - val_accuracy: 0.9613 - v

In [20]:
### Langkah 6: Evaluasi dan Simpan Model
model.evaluate(X_test, y_test)
model.save('gesture_to_text_model.h5')

27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.9831 - loss: 0.0560


In [21]:
### Langkah 7: Menggunakan Model untuk Prediksi
model = tf.keras.models.load_model('gesture_to_text_model.h5')

def predict_gesture(image_path, model):
    image = cv2.imread(image_path)
    image = cv2.resize(image, (64, 64))
    image = image / 255.0
    image = np.expand_dims(image, axis=0)
    prediction = model.predict(image)
    predicted_label = np.argmax(prediction)
    return chr(predicted_label + ord('A'))

In [26]:
model = tf.keras.models.load_model('gesture_to_text_model.h5')
def predict_gesture_in_folder(folder_path, model):
    predicted_gestures = []
    for file_name in os.listdir(folder_path):
        if file_name.lower().endswith(('.png', '.jpg', '.jpeg')):
            image_path = os.path.join(folder_path, file_name)
            predicted_gesture = predict_gesture(image_path, model)
            predicted_gestures.append(predicted_gesture)
    return predicted_gestures



In [32]:
model = tf.keras.models.load_model('gesture_to_text_model.h5')
def predict_video(video_path, model):
    cap = cv2.VideoCapture(video_path)
    predicted_gestures = []

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        
        # Ubah ukuran bingkai menjadi 64x64 (jika diperlukan)
        frame = cv2.resize(frame, (64, 64))

        # Normalisasi bingkai
        frame = frame / 255.0

        # Tambahkan dimensi batch (batch dimension) ke bingkai
        frame = np.expand_dims(frame, axis=0)

        # Lakukan prediksi untuk bingkai
        prediction = model.predict(frame)
        predicted_label = np.argmax(prediction)
        predicted_gestures.append(chr(predicted_label + ord('A')))
    
    cap.release()

    return predicted_gestures

In [33]:
# Contoh penggunaan
video_path = 'C:/Users/askas/Downloads/Dataset/A/Azab.webm'
predicted_gestures = predict_video(video_path, model)
print("Predicted Gestures:", predicted_gestures)

Predicted Gestures: []


In [25]:
# Contoh penggunaan
image_path = 'C:/Users/askas/Downloads/Dataset/A/Azab'
predicted_gesture = predict_gesture(image_path, model)
print(f"Predicted Gesture: {predicted_gesture}")

error: OpenCV(4.9.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\resize.cpp:4152: error: (-215:Assertion failed) !ssize.empty() in function 'cv::resize'


In [29]:
# Contoh penggunaan
folder_path = 'C:/Users/askas/Downloads/Datamentah/A/Azab'
predicted_gestures = predict_gesture_in_folder(folder_path, model)
print(f"Predicted Gestures: {predicted_gestures}")

FileNotFoundError: [WinError 3] The system cannot find the path specified: 'C:/Users/askas/Downloads/Datamentah/A/Azab'